## Levatamos Fermionic

In [1]:
import numpy as np
import openfermion as of
from tqdm import tqdm
from itertools import combinations
from openfermion.utils import commutator, count_qubits, hermitian_conjugated
import functools
import concurrent.futures
from numba import njit
import tensorflow as tf
import scipy
import sparse
import itertools
import linecache
from multiprocessing import Pool, cpu_count

# Generación de base
class fixed_basis:
    @staticmethod
    def int_to_bin(k, d):
        return np.base_repr(k, 2).zfill(d)

    @staticmethod
    def bin_to_op(b):
        tups = [(i, 1) for i, k in list(enumerate(list(b))) if k == '1']
        return of.FermionOperator(tups)

    def idx_to_repr(self, idx):
        return self.canonicals[idx]

    def opr_to_idx(self, opr):
        for i in range(self.size): # Evitar esto ordenando opr
            if self.base[i] == opr:
                return i

    # Calcula el valor medio a partir del indice del vector y el operador
    def idx_mean_val(self, idx: int, op: of.FermionOperator):
        vec = self.idx_to_repr(idx)
        return np.real(np.transpose(vec) @ of.get_sparse_operator(op, n_qubits=self.d) @ vec)

    # Calcula el valor medio a partir de un estado y el operador
    def mean_val(self, vec, op):
        idx = self.opr_to_idx(vec)
        return self.idx_mean_val(idx, op)

    # Calcula la contracción de un operador sobre dos estados dados
    def idx_contraction(self, idx_1, idx_2, op):
        rep = lambda x: self.idx_to_repr(x)
        return np.real(np.transpose(rep(idx_1)) @ of.get_sparse_operator(op, n_qubits=self.d) @ rep(idx_2))

    def create_basis(self, d, num = None, pairs = False):
        basis = []
        num_ele = []
        for k in range(0,2**d):
            b = self.int_to_bin(k, d)
            if num != None:
                if b.count('1') == num:
                    if pairs:
                        if np.all(b[::2] == b[1::2]):
                            oper = self.bin_to_op(b)
                            basis.append(oper)
                            num_ele.append(k)
                    else:
                        oper = self.bin_to_op(b)
                        basis.append(oper)
                        num_ele.append(k)
            else:
                oper = self.bin_to_op(b)
                basis.append(oper)
        return basis, num_ele

    def __init__(self, d, num = None, pairs = False, basis = None, num_ele = None):
        self.d = d
        self.num = num
        self.m = num
        # Si nos da la base, la levantamos (asumimos GC). Si no, la creamos
        if basis is None:
            self.base, self.num_ele = self.create_basis(d, num, pairs)
        else:
            self.base, self.num_ele = basis, num_ele
        self.size = len(self.base)
        self.canonicals = np.eye(self.size)
        self.pairs = pairs

    @staticmethod
    def cdc(i, j):
        return of.FermionOperator(((i,1),(j,0)))

    @staticmethod
    def cc(i, j):
        return of.FermionOperator(((i,0),(j,0)))

    # Del indice, cuenta el número de partículas
    def num_idx(self, idx):
        b = self.int_to_bin(idx, basis.d)
        return b.count('1')

    # Calculo de rho1 (via directa, lento, y solo definido en la base por ahora)
    def rho_1(self, op):
        # Necesitamos un índice, es?
        if type(op) != int:
            op = self.opr_to_idx(op)
        mat = np.zeros((self.d, self.d))
        for i in range(self.d):
            for j in range(self.d):
                cdc = self.cdc(j, i)
                mat[i,j] = self.idx_mean_val(op, cdc)
        return mat

# Calculo de generadores de rho1
def rho_1_gen(basis):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    indices = []
    values = []
    shape = (basis.d, basis.d, basis.size, basis.size)
    d = basis.d
    for i in tqdm(range(0, d)):
        for j in range(0, d):
            # Generamos el operador
            op = basis.cdc(j, i)
            #print(op)
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# Calculo de rho1 (via generadores) de un vector en la base canonica
def rho_1(vect, rho_1_arrays):
    if len(vect.shape) == 1: # vectores
        return sparse.einsum('k,ijkl,l->ij', vect, rho_1_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad
        return sparse.einsum('ijkl,kl->ij', rho_1_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_1_arrays)

# Calculo de indices de rho2kkbar
def get_kkbar_indices(t_basis):
    indices = []
    for i, ind in enumerate(t_basis.num_ele):
        v = t_basis.int_to_bin(ind, t_basis.d)
        if np.all(v[::2] == v[1::2]):
            indices.append(i)
    return indices

# Calculo de generadores de rho2
def rho_2_gen(basis, t_basis, idx_list = []):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    d = basis.d
    indices = []
    values = []
    if len(idx_list) == basis.m:
        idx_list = idx_list
    elif len(idx_list) == basis.m**4:
        idx_list = np.unique(idx_list[:,0])
    else:
        idx_list = range(t_basis.size)
    shape = (len(idx_list), len(idx_list), basis.size, basis.size)
    for i, ii in tqdm(enumerate(idx_list), total=len(idx_list)):
        for j, jj in enumerate(idx_list):
            # Generamos el operador
            op = t_basis.base[jj]*of.utils.hermitian_conjugated(t_basis.base[ii])
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)

    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# rho_m_gen aux func
def process_chunk(args):
    chunk, m_basis, basis, d, it_set = args
    indices = []
    values = []
    for ii in chunk:
        for jj in it_set:
            # Generate the operator
            op = m_basis.base[jj] * of.utils.hermitian_conjugated(m_basis.base[ii])
            mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extract the information
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([ii, jj, r, c])
                values.append(v)
    return indices, values

# Parallelized rho_m_gen
def rho_m_gen(basis, m, num_workers=None):
    if num_workers is None:
        num_workers = cpu_count()  # Use all available CPUs by default
    
    indices = []
    values = []
    m_basis = fixed_basis(basis.d, num=m, pairs=basis.pairs)
    shape = (m_basis.size, m_basis.size, basis.size, basis.size)

    it_set = np.arange(m_basis.size)
    chunks = np.array_split(it_set, num_workers)  # Split `it_set` into chunks for each worker

    # Use multiprocessing Pool for parallel processing
    with Pool(processes=num_workers) as pool:
        # Pass arguments as tuples instead of using a lambda
        results = list(
            tqdm(
                pool.imap(
                    process_chunk, 
                    [(chunk, m_basis, basis, basis.d, it_set) for chunk in chunks]
                ),
                total=num_workers
            )
        )
    
    # Collect results from all processes
    for indices_chunk, values_chunk in results:
        indices.extend(indices_chunk)
        values.extend(values_chunk)

    # Construct the sparse array
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

def rho_m(vect, rho_m_arrays):
    return sparse.einsum('k,ijkl,l->ij', vect, rho_m_arrays, vect)

# Calculo de rho2 (via generadores) de un estado en la base canonica
def rho_2(vect, rho_2_arrays):
    if len(vect.shape) == 1: # vectores SOLO RHO2 COMPLETA
        return sparse.einsum('k,ijkl,l->ij', vect, rho_2_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad SOLO RHO2 COMPLETA
        return sparse.einsum('ijkl,kl->ij', rho_2_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_2_arrays)

# Calculo de generadores de K (usado para quasiparticles) WIP SPARSE
def k_gen(basis):
    mat = np.zeros((basis.d, basis.d, basis.size, basis.size))
    d = basis.d
    for i in tqdm(range(0, d), total=d):
        for j in range(0, d):
            op = basis.cc(j, i)
            if basis.num == None:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
            else:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]
    return mat

def k_vect(vect, k_gen):
    return np.einsum('k,ijkl,l->ij', vect, k_gen, vect)

# Calculo la matrix rho de cuasipartículas  WIP SPARSE
def rho_qsp(vect, rho_1_arrays, k_arrays, rho1 = None):
    if type(rho1) == None:
        rho1 = rho_1(vect, rho_1_arrays)
    k = k_vect(vect, k_arrays)

    mat = np.block([[rho1, k], [-np.conjugate(k), np.eye(rho_1_arrays.shape[0])-np.conjugate(rho1)]])
    return mat

# Devuelve los indices que tienen a level ocupado
def level_proy(d, level):
    ids = []
    for k in range(0,2**d):
        b = fixed_basis.int_to_bin(k, d)
        if b[level] == '1':
            ids.append(k)
    arr = np.zeros(2**d)
    arr[np.array(ids)] = 1
    return arr, ids

def parity_levels(d):
    rng = range(2**d)
    binary_repr = np.vectorize(np.binary_repr)(rng)
    ones_c = np.char.count(binary_repr, '1')
    return np.array(rng)[ones_c % 2 == 1] # seleccionamos estados impares

# Devuelve el vector postmedido
def measure(basis, vect, level = 1):
    l_arr, l_ids = level_proy(basis.d, level)
    proy_v = vect * l_arr
    comp_arr = np.logical_not(l_arr).astype(int)
    comp_v = vect * comp_arr
    norm = lambda v: v / np.linalg.norm(v)
    return norm(proy_v), norm(comp_v)

def entropy(rho, m):
    S_fun = lambda rho: -1*np.trace(rho @ scipy.linalg.logm(rho)) / np.log(2)
    ent = S_fun(rho) / (np.log2(scipy.special.binom(basis.d, m)))
    return ent

# Levanta bases de QChem
def build_csv_basis(csvf, d):
    # Construimos la base
    ops = []
    with open(csvf, 'r') as basis:
        num_ele = []
        # Contar los niveles
        m_level = 0
        # Creamos operadores
        for l in basis.read().splitlines()[4:]:
            natop = [int(x) for x in l.split(' ')[1:]] # Operador en forma de lista
            #print(natop)
            op = of.FermionOperator(([(i, 1) for i in natop]))
            ops.append(op)
            # Contamos niveles
            m_level = max(m_level, *natop)
            # Determinamos el índice
            natop_to_int = lambda x: np.sum([2**(d-1-i) for i in x])
            num_ele.append(natop_to_int(natop))
 
        # Determinamos m y d
        num = len(natop)
        #assert d == m_level+1

    return fixed_basis(d, num = num, pairs = False, basis = ops, num_ele=num_ele)

2025-01-20 11:02:36.097164: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-20 11:02:36.097405: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-20 11:02:36.155963: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 11:02:36.286340: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-20 11:02:37.323117: W tensorflow/compiler/tf2

Debido al número de niveles (innecesarios) en el core, el objetivo es calcular la base natural (a partir de rho1), y proyectar el fundamental sobre el subespacio generado por los estados no ocupados. Allí, buscamos la descomposición, y luego volvemos a la base original

In [2]:
# Aux func
def remove_null_terms(op):
    op_nn = of.FermionOperator.zero()
    terms = list(op.terms.items())
    act_idx = lambda tt: np.array([i[0] for i in tt[0]])
    filtered_terms = []

    for term in terms:
        acts = np.array(act_idx(term))
        u, c = np.unique(acts, return_counts=True)
        if np.max(c) == 1:
            filtered_terms.append(term)

    return filtered_terms

def op_to_rep(basis, op): # CHEQUEADO
    op = -1*of.transforms.normal_ordered(op)
    terms = list(op.terms.items())
    act_idx = lambda tt: [i[0] for i in tt[0]]
    vect = np.zeros(basis.size)

    for term in terms:
        act = act_idx(term)
        nele = np.sum([2**(basis.d-1-i) for i in act]) # si
        nele_idx = np.argwhere(basis.num_ele == nele)[0][0]
        vect[nele_idx] += term[1]
    
    return vect
    
def build_basis_from_vect(basis, vect, tol = 1e-10):
        ops = []
        num_ele = []
        vect_pro = []
        for idx, coord in enumerate(vect):
                if np.abs(coord) > tol:
                        ops.append(basis.base[idx])
                        num_ele.append(basis.num_ele[idx])
                        vect_pro.append(vect[idx])
        return fixed_basis(basis.d, num = basis.num, pairs = False, basis = ops, num_ele=num_ele), np.array(vect_pro)


In [3]:
# Dado la base, una rotación entre los 1SP states y un vector (representado en basis)
# devuelve la representación en la base natural
def natural_basis_rotation(basis, C, vect):
    # Calculamos la transformación de cada generador
    targ_sp = []
    for i in range(basis.d):
        op = np.sum([of.FermionOperator(((basis.d-1-j,1)),C[i,j]) for j in range(basis.d) if np.abs(C[i,j]) > 1e-5])
        targ_sp.append(op)

    targ_sp.reverse()

    # Reemplazamos en el fundamental
    vect_op = np.sum([vect[i] * basis.base[i] for i in range(len(vect))])
    terms = list(vect_op.terms.items())

    vect_op_t = of.FermionOperator.zero()
    for term in terms:
        # Construimos el término
        act_idx = lambda tt: [i[0] for i in tt[0]]
        tt = np.prod([targ_sp[i] for i in act_idx(term)])
        vect_op_t += term[1] * tt
        
    # Filtramos los términos nulos
    f_terms = remove_null_terms(vect_op_t)
    vect_op_ft = of.FermionOperator.zero()
    for ft in f_terms:
        vect_op_ft += of.FermionOperator(ft[0],ft[1])

    # Calculamos la representación en la base
    ext_basis = fixed_basis(basis.d, basis.m)
    res = op_to_rep(ext_basis, vect_op_ft)
    res = 1/np.linalg.norm(res) * res

    return res, vect_op_ft

### Determinación de estado

##### Cargamos el input

In [62]:
# Levantamos la base, y calculamos rho1
basis = build_csv_basis('h2o_basis', 16)
rho_1_arrays = rho_m_gen(basis, 1, num_workers=32)

# Input
rho_1_obj  = np.sort(np.concatenate([np.repeat(1/2, 2*4), np.repeat(1, 2*3), np.repeat(0, 2)]))
rho_2_obj = np.sort(np.concatenate([np.repeat(1/12, 2*4+4), np.repeat(1/3, 2*2+2), np.repeat(1/2, 2*12+24), np.repeat(1, 2*3+9), np.repeat(3/4, 4), np.repeat(0,35)]))
vect = [9.493212998391736e-03,-1.403538055609714e-11,4.649311768875863e-02,-6.567918880740946e-02,-1.367081630226763e-10,2.842633289527116e-01,5.151134821828296e-11,3.993086414501282e-10,-9.149353837339585e-11,-1.090929564964267e-10,3.949485912946310e-04,-2.316418934376388e-11,1.317140230653304e-04,1.316576192646498e-11,-3.474844023562381e-11,3.178771809067837e-06,4.573718498017584e-12,5.709752739953042e-07,2.504287778193379e-13,-6.789048360945855e-12,-6.852060660027462e-13,-3.708988639227866e-13,-2.101698089571679e-03,1.183041653354087e-11,-9.911458144575837e-12,-2.881444405235761e-01,1.906978136701541e-10,-7.011195776675775e-11,-3.121319628580168e-12,-2.889519071965350e-10,-1.539700811647047e-11,-4.678246291769944e-14,7.980115771668993e-05,6.751588770236491e-12,-1.176018660778170e-03,-3.075727056073582e-11,1.501555624120258e-12,1.472217770578803e-06,2.044056393055121e-12,-8.018130190795768e-06,-2.673141167920912e-12,-5.660998493171163e-07,4.649311769624395e-02,-1.330002514269370e-11,-4.085589864474602e-03,-5.724077699943703e-01,4.525109675729083e-12,-3.683260995838695e-02,-3.255138565508400e-11,1.596913298463504e-11,1.015384816998776e-11,-4.311036078608247e-10,-5.191287392693676e-05,2.440384045890194e-12,-9.379430620967329e-06,-5.086657071651381e-12,-3.953235213612016e-12,9.012536938807939e-07,2.413080242109217e-11,4.451000592944688e-08,8.440639940758612e-13,1.059129092386257e-12,3.183821053453346e-13,-6.567918878210724e-02,-1.608577221023579e-11,-5.724077699252504e-01,-3.893336102393095e-03,4.132197172862604e-11,2.439543883438981e-02,-7.855931486939994e-11,1.078910509956508e-10,3.791852423671252e-11,-4.766031539351433e-11,3.368902220766659e-04,-8.872418158033230e-12,-2.559675963915414e-03,-4.235788056935718e-12,2.311268906192012e-11,1.729098243347753e-06,-1.778453400643153e-12,-1.771394689130530e-05,-3.981863326088516e-12,-7.583894918827121e-14,5.605006910150374e-13,1.981261165356218e-10,-2.881444404063113e-01,-2.205507036725249e-11,8.949121048898374e-11,-2.116501902342333e-03,2.953466712904848e-11,2.015095354213556e-11,-1.055658174573602e-10,1.058507701748906e-10,4.620328469195291e-11,1.356490364565209e-11,-3.386785383813768e-05,-5.581608287195181e-12,4.092631961210984e-04,8.304836246917870e-11,2.015265937260744e-14,-6.489416735907605e-07,1.840756831311177e-12,3.094976851695781e-06,-1.661379435853226e-12,7.281742312398216e-07,2.842633288916130e-01,-1.702009403214580e-10,-3.683260994106137e-02,2.439543870803070e-02,6.082078469446301e-11,4.434738531532339e-03,4.786322557559264e-12,-7.433089232105201e-11,1.703641753125092e-11,-8.636299039635753e-11,-1.383657139662060e-03,-8.794354432299964e-12,-2.155452688294493e-04,-3.639821351448870e-12,2.230306474810379e-11,-9.695831549938936e-06,-8.089267844771269e-12,-8.937754195833808e-07,-1.576132480748231e-12,-8.982501010282569e-12,2.258817000973842e-12,2.711141259266517e-11,4.691185691539211e-11,1.580148539311515e-10,7.471597254440476e-11,7.733600917358643e-12,3.141828876886342e-11,-9.771715106263039e-05,-2.917047133464889e-11,6.987274520950660e-04,-3.683578842639505e-12,-1.334306117653743e-12,4.554940116944121e-11,-3.231284023738668e-11,-2.150333406633041e-11,4.124130511079267e-12,5.041321699800213e-12,1.588021701518278e-12,-4.498156038441317e-14,-4.123723763190693e-13,-5.525590241351556e-12,-2.930774648604007e-13,-1.515010731684398e-10,7.370117607400919e-11,-4.618576710602899e-11,-5.949391889557398e-11,2.878607101972053e-11,-1.187345800843341e-10,-6.864414034677405e-11,-1.206123558002822e-04,-1.488676014443912e-12,9.065512487160873e-04,-3.379909745999564e-11,1.910005721038195e-12,1.021653641565369e-11,-8.269130076559394e-11,-2.798646810264202e-07,-5.102997572784153e-12,-4.079735386331037e-13,1.658836241008379e-12,2.482969068287357e-12,1.695907393466894e-09,3.821899991708097e-13,-1.460747825094933e-10,1.682705391649173e-10,2.008114829016675e-11,-2.202180973419363e-12,1.056890517736124e-10,-9.178714306556471e-11,6.987275830686379e-04,-4.238750717550655e-11,3.701020767801657e-05,-5.012045738722695e-12,-2.917013910416504e-11,1.302966572864761e-11,1.457070612891414e-12,-2.289493377844331e-11,9.066182723149432e-13,1.468552403515558e-12,-2.662864904259473e-12,1.110304981202208e-12,6.995392256168789e-12,1.556038145430544e-12,2.693996600998102e-12,9.385979371346961e-11,2.946204170431151e-11,4.018686872322880e-10,4.974966591987190e-11,7.373070805196919e-12,4.732591872341746e-11,-2.144324615568012e-12,9.065512592688823e-04,2.286146876279055e-11,6.271437827072584e-05,3.000790108864038e-11,3.935940368002065e-11,-1.028884126148764e-10,-3.271619839682315e-13,4.672154009973494e-06,1.064984441258250e-13,6.775662515487354e-12,5.219095623426322e-13,3.009162970121251e-12,2.078308693186735e-09,9.601464343863923e-13,3.949485881660820e-04,-7.071736649649897e-13,-5.191287693767832e-05,3.368902271494889e-04,6.252292489854172e-12,-1.383657177176408e-03,-3.805121232261181e-12,-5.925886662070161e-11,2.913548017927870e-11,-2.642356151361688e-11,-1.363369411807896e-04,5.313453102861247e-11,9.361668378587847e-04,-2.030618694806490e-12,-2.754669091392762e-11,-3.331617776393535e-06,3.555043054432922e-12,2.258233636115107e-05,3.791817105882217e-12,2.097493274879215e-12,1.338279764826437e-11,-2.353523619040800e-11,7.980114433584928e-05,1.626053908268813e-12,-1.291542223362888e-11,-3.386785278476000e-05,-1.452474232336886e-11,5.286687458194194e-11,-9.644090596519690e-12,3.803433867804991e-12,8.493266012316513e-11,2.191815702023751e-11,-1.528804299097781e-04,-4.758142571154899e-12,1.217603154089707e-03,-3.475523543348794e-11,-1.331194529391727e-13,-3.546006687189228e-06,-3.574150156802156e-12,3.015844899497634e-05,3.533940636854416e-14,1.034151357848499e-08,1.317140411159087e-04,7.926588489652809e-12,-9.379430818779136e-06,-2.559675998606472e-03,-1.068968966337307e-12,-2.155452692971603e-04,3.765567587122260e-11,-1.283232680861196e-12,1.154142487868370e-11,2.926839686731790e-11,9.361669120784761e-04,7.551701208919833e-12,6.031888738333188e-05,-1.587990088270413e-11,-2.501078394523150e-11,2.314842860627909e-05,-1.453708568581687e-11,1.668821631546635e-06,-1.564812957356815e-12,-1.441580567645387e-12,-1.283264896269941e-12,5.798278818234023e-12,-1.176018692844553e-03,-4.177047733730145e-13,-4.096907678023554e-12,4.092631938851082e-04,-1.526264637089188e-12,2.417147398445785e-11,-4.264675448083156e-11,9.034250635273577e-11,3.012594617262705e-12,8.164709805178823e-12,1.217603227851483e-03,-2.817895977628923e-11,8.610888194951957e-05,1.745482714529611e-11,3.341069184854576e-12,2.943028189878144e-05,-9.180229094586087e-12,1.901594872598002e-06,-4.739636748238441e-12,-1.350338357238904e-07,-5.754530284674729e-11,3.147617049191186e-11,5.441409259043753e-12,-1.820740263561924e-11,4.126576629831305e-11,-9.106581290277156e-11,4.269252583401780e-12,-2.798622832198465e-07,2.645437757541423e-12,4.672154794684845e-06,3.328911068623947e-11,9.474860954853064e-12,-1.830316216329921e-11,-1.721222489783294e-11,2.491825223985215e-06,1.872420343640541e-12,3.007645571253803e-12,4.581611659729421e-12,-5.762895510942690e-12,7.073125433767267e-08,-1.778278505767700e-11,3.178771632937198e-06,-6.166813451022855e-13,9.012373540363324e-07,1.729089315050892e-06,1.710841450951147e-12,-9.695820657400887e-06,4.049814970814497e-12,4.638941868994780e-14,2.230817396501688e-12,1.041491220174692e-12,-3.331632910515570e-06,2.100971480588163e-12,2.314843015097350e-05,3.381032918919892e-12,2.354331187450128e-12,-1.124491752228453e-06,1.885552793007390e-13,7.661062364167973e-06,-1.303806248043920e-12,2.567289481525267e-13,3.232082707462361e-13,1.570104053622796e-12,1.472216898820490e-06,4.759281655196799e-12,1.745888544021612e-12,-6.489534263549564e-07,-6.932466374996649e-12,6.799722301245292e-12,1.103088791174978e-12,-2.198729275828566e-12,-4.693159461937418e-12,-4.094199158636476e-13,-3.545999162650055e-06,-9.074681938794369e-12,2.943027527303349e-05,-6.125659451997191e-13,3.892952451363958e-13,-1.185913941472911e-06,-8.390766909912208e-12,9.979081294191579e-06,-8.895126642667612e-13,1.547958929908616e-09,5.709735244694984e-07,3.132599983135356e-12,4.447845396097917e-08,-1.771393609244976e-05,1.484506825102543e-12,-8.937607971638028e-07,1.896246770714371e-13,2.241889133980663e-12,-3.122048387879676e-13,-8.076631786585428e-12,2.258232654132016e-05,-9.565561231006300e-12,1.668819913228741e-06,-3.926234288660488e-12,-5.003392930411755e-12,7.661061946477902e-06,-1.250460891119836e-12,5.610873669299694e-07,-2.549389073918289e-13,-7.484633322018630e-14,-1.044141778042896e-13,4.802241532594192e-13,-8.018120089263053e-06,-1.101132863174038e-13,-5.769225105356123e-12,3.094976769495003e-06,-8.389591457880947e-13,1.143870151223206e-13,-6.129272792248973e-12,-3.758234653621819e-13,1.129527179184200e-12,2.670323233404351e-12,3.015845243207793e-05,-4.139096661005171e-12,1.901579945366051e-06,-6.073866728911566e-12,1.587988307492750e-12,9.979081383791410e-06,-3.917282076087210e-13,6.491902085189736e-07,1.007825572251306e-12,-3.744703496738376e-08,-3.065656755727601e-12,2.300041291901508e-12,-4.667737963350885e-13,-3.813102756310100e-13,4.169866484108828e-12,7.326620348943926e-13,-6.852660009364331e-12,1.698686704891940e-09,-3.302936810723410e-12,2.086421853981982e-09,2.158789123362150e-12,1.070962561358965e-12,1.892740499914526e-12,-4.687290498075707e-12,7.071428843300519e-08,-9.187552029227700e-14,-1.000809431727905e-12,1.413417987005012e-13,-1.502268029819976e-12,2.371709104846517e-08,1.201679344637095e-12,1.344554250405245e-12,-5.660946713452370e-07,-1.256097441183135e-12,-8.424982262765184e-13,7.281701183283107e-07,5.987617166368957e-12,2.473264030192610e-13,-1.362107012248053e-12,-2.715542169392558e-12,1.403706823480920e-12,-1.021323163148066e-11,1.034705288291469e-08,-8.677063551013730e-12,-1.349930883050591e-07,-2.783739631609858e-11,-1.996018279281773e-13,1.547462714946436e-09,-9.559044750128227e-13,-3.744628031553583e-08,-1.354287703197667e-12,3.334749975749791e-08]
vect = np.array(vect, dtype=np.float32)

op_fund = np.sum([vect[i] * basis.base[i] for i in range(basis.size)])


100%|██████████| 32/32 [00:07<00:00,  4.02it/s]


In [63]:
# Cambio a base natural 
rho_1 = rho_m(vect, rho_1_arrays).todense()
evals, evects = scipy.linalg.eigh(rho_1)
C = evects
vect_pro, op_pro = natural_basis_rotation(basis, C, vect)
# Calculamos la base extendida, donde vive vect_pro y restringimos
ext_basis = fixed_basis(basis.d, basis.m)
basis_pro, vect_pro = build_basis_from_vect(ext_basis, vect_pro)
# Calculamos los arrays en esta base
rho_1_arrays = rho_m_gen(basis_pro, 1, num_workers=32)
rho_2_arrays = rho_m_gen(basis_pro, 2, num_workers=32)

100%|██████████| 32/32 [07:16<00:00, 13.64s/it]  


In [89]:
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.cluster.vq import kmeans, vq

# Aux functions
r_eig = lambda x: np.sort(np.real(np.linalg.eigvals(x.todense())))
k = 1 # Número de clusters

def sparsity_error(A, k=2):
    # L1
    sparsity_term = np.sum(np.abs(A))

    # Clustering 
    non_zero_coeffs = A[np.abs(A) > 1e-5]  # Sacamos los ceros
    if len(non_zero_coeffs) > 0:
        # Calculamos los clusters (k) y la distancia en ellos
        centroids, _ = kmeans(non_zero_coeffs, k)
        cluster_labels, distances = vq(non_zero_coeffs, centroids)
        cluster_error = np.sum(distances**2) 
    else:
        cluster_error = 0

    loss = 1 * sparsity_term + 2 * cluster_error
    return loss

# Loss definition
def loss_fun(vect):
    # Rho 2 loss
    rho = rho_m(vect, rho_2_arrays)
    eigv = r_eig(rho)
    lrho2 = np.linalg.norm(rho_2_obj-eigv)
    # Rho 1 loss
    rho = rho_m(vect, rho_1_arrays)
    eigv = r_eig(rho)
    lrho1 = np.linalg.norm(rho_1_obj-eigv)
    # L1 + clustering loss
    l1 = sparsity_error(vect, k) 

    print(lrho2, lrho1, 1/10 * l1)
    return lrho2 + lrho1 + 0.1 * l1

opt = scipy.optimize.minimize(loss_fun, vect_pro, method='L-BFGS-B')

0.15577714636272638 0.06343442938019345 0.4716420312601619
0.1557771463629162 0.06343442938030312 0.4716420302601618
0.15577714636272816 0.06343442938019324 0.47164203226016177
0.1557771463627286 0.06343442938019309 0.47164203226016177
0.15577714636291595 0.06343442938030378 0.4716420302601618
0.1557771463627281 0.06343442938019314 0.4716420302601618
0.1557771463627284 0.06343442938019343 0.4716420302601618
0.1557771463625459 0.06343442938007665 0.4716420302601618
0.15577714636254705 0.06343442938007597 0.4716420302601618
0.1557771463627286 0.06343442938019352 0.4716420302601618
0.1557771463627277 0.06343442938019389 0.47164203226016177
0.1557771463627281 0.06343442938019338 0.4716420302601618
0.15577714636272838 0.06343442938019356 0.47164203226016177
0.1557771463627623 0.0634344293802139 0.4716420302601618
0.15577714636272683 0.06343442938019321 0.4716420302601618
0.15577714636272794 0.06343442938019349 0.47164203226016177
0.155777146362759 0.06343442938021203 0.4716420302601618
0.15

In [91]:
print(opt)
print(opt.x)
print(sparsity_error(opt.x, k=k))
print(rho_1_obj)
np.round(r_eig(rho_m(opt.x, rho_1_arrays)),2)

  message: STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
  success: False
   status: 1
      fun: 0.4762340907669579
        x: [ 6.049e-05  1.469e-06 ...  1.267e-05  4.330e-01]
      nit: 50
      jac: [ 8.922e-02  1.000e-01 ... -2.078e-03  4.075e+00]
     nfev: 18396
     njev: 146
 hess_inv: <125x125 LbfgsInvHessProduct with dtype=float64>
[ 6.04858128e-05  1.46942578e-06  1.48063400e-06  5.74539475e-05
  6.70741552e-05  6.70867181e-05  6.22170274e-05  6.35247734e-05
  6.52430450e-05  2.69439483e-07  6.50142337e-05  4.90202490e-07
  6.64891283e-05  6.72162692e-05  6.88541854e-07  6.49931558e-05
  6.76528841e-05  1.71607156e-06  7.01031654e-05  6.94649293e-05
  1.76614811e-06  1.97636138e-06  6.18160092e-05  6.58329173e-05
  6.16427021e-05  6.90442963e-05  6.44890899e-05  6.39657932e-05
  6.52620029e-05  6.54723688e-05 -5.30111366e-07  6.46595957e-05
  1.75940152e-06  1.29361492e-06  3.45657681e-07  1.02641259e-06
  9.64935113e-07  4.73238346e-07  1.72982532e-06  1.94635405e-0

array([0. , 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 1. , 1. ,
       1. , 1. , 1. ])

In [92]:
final_res = of.FermionOperator.zero()

tol = 1e-3
for idx, coord in enumerate(opt.x):
    final_res += basis_pro.base[idx] * coord if np.abs(coord) > tol else 0
    if np.abs(coord) > tol:
        print((1/np.abs(coord)**2))

final_res, unique_ele(final_res)


5.333333796814192
5.333302549706838
12.000044226101577
48.00218300991128
47.99982828129923
47.999777217361036
48.00212305308927
12.00004446667475
5.3332978641037725
5.3333297099491475


(0.4330128489833608 [0^ 1^ 2^ 3^ 4^ 5^ 6^ 7^ 8^ 9^] +
 0.4330141417706756 [0^ 1^ 2^ 3^ 4^ 5^ 6^ 7^ 10^ 11^] +
 0.2886745997453277 [0^ 1^ 2^ 3^ 4^ 5^ 6^ 8^ 10^ 12^] +
 0.1443343753583106 [0^ 1^ 2^ 3^ 4^ 5^ 6^ 8^ 11^ 13^] +
 -0.14433790225590726 [0^ 1^ 2^ 3^ 4^ 5^ 6^ 9^ 10^ 13^] +
 -0.14433782547996926 [0^ 1^ 2^ 3^ 4^ 5^ 7^ 8^ 11^ 12^] +
 0.14433428521831473 [0^ 1^ 2^ 3^ 4^ 5^ 7^ 9^ 10^ 12^] +
 0.2886746026389572 [0^ 1^ 2^ 3^ 4^ 5^ 7^ 9^ 11^ 13^] +
 0.4330139515571243 [0^ 1^ 2^ 3^ 4^ 5^ 8^ 9^ 12^ 13^] +
 0.4330126830772425 [0^ 1^ 2^ 3^ 4^ 5^ 10^ 11^ 12^ 13^],
 {0, 1, 2, 3, 4, 5})

15.99

#### Testing

Verificamos que los eig de rho1 se preservan

In [44]:
# Cálculo de base natural
rho_1 = rho_m(vect, rho_1_arrays).todense()
evals, evects = scipy.linalg.eigh(rho_1)
C = evects

res, vect_op_ft = natural_basis_rotation(basis, C, vect)

ext_basis = fixed_basis(basis.d, basis.m)
rho_1_arrays_n = rho_m_gen(ext_basis, 1, num_workers=32)
rho_1_n = rho_m(res, rho_1_arrays_n).todense()
print(np.trace(rho_1_n), np.trace(rho_1))

evv = lambda vv: np.sort(np.real(np.linalg.eigvals(vv)))
print(evv(rho_1), evv(rho_1_n))

rep_to_op = lambda vect: np.sum([vect[i] * basis.base[i] for i in range(len(vect))])
#unique_ele(vect_op_ft), unique_ele(rep_to_op(vect))

ValueError: Inconsistent shape for index 'k'.

Verificamos que la composición, es la identidad

In [94]:
res, vect_op_ft = natural_basis_rotation(basis, C, vect)
res2, vect_op_ft2 = natural_basis_rotation(ext_basis, C.T, res)

vect_op = np.sum([vect[i] * basis.base[i] for i in range(len(vect))])

of.transforms.normal_ordered(vect_op_ft2), res2, of.transforms.normal_ordered(vect_op)

(-0.009493221505521588 [11^ 10^ 9^ 8^ 7^ 4^ 3^ 2^ 1^ 0^] +
 0.06567915392863352 [11^ 10^ 9^ 8^ 7^ 5^ 4^ 2^ 1^ 0^] +
 -0.00039479040463999686 [11^ 10^ 9^ 8^ 7^ 5^ 4^ 3^ 2^ 0^] +
 -3.372243029427183e-06 [11^ 10^ 9^ 8^ 7^ 5^ 4^ 3^ 2^ 1^] +
 -0.046493134167591176 [11^ 10^ 9^ 8^ 7^ 6^ 3^ 2^ 1^ 0^] +
 -0.28426333153569405 [11^ 10^ 9^ 8^ 7^ 6^ 5^ 2^ 1^ 0^] +
 -0.00013174919476567257 [11^ 10^ 9^ 8^ 7^ 6^ 5^ 3^ 2^ 0^] +
 3.4219639201432875e-07 [11^ 10^ 9^ 8^ 7^ 6^ 5^ 3^ 2^ 1^] +
 0.002101619231354521 [12^ 10^ 9^ 8^ 7^ 5^ 3^ 2^ 1^ 0^] +
 0.2881444085777637 [12^ 10^ 9^ 8^ 7^ 6^ 4^ 2^ 1^ 0^] +
 -7.974636812387609e-05 [12^ 10^ 9^ 8^ 7^ 6^ 4^ 3^ 2^ 0^] +
 -2.402101031541447e-06 [12^ 10^ 9^ 8^ 7^ 6^ 4^ 3^ 2^ 1^] +
 0.0011760974915812092 [12^ 10^ 9^ 8^ 7^ 6^ 5^ 4^ 2^ 0^] +
 8.012654141343571e-06 [12^ 10^ 9^ 8^ 7^ 6^ 5^ 4^ 2^ 1^] +
 5.654427524408164e-07 [12^ 10^ 9^ 8^ 7^ 6^ 5^ 4^ 3^ 2^] +
 0.06567916523926123 [12^ 11^ 9^ 8^ 7^ 4^ 3^ 2^ 1^ 0^] +
 0.00389332162644885 [12^ 11^ 9^ 8^ 7^ 5^ 4^ 2^ 1^ 0^] +


Funciones auxiliares

In [49]:
def unique_ele(op):
    terms = list(op.terms.items())
    act_idx = lambda tt: [i[0] for i in tt[0]]
    terms_set = []
    for term in terms:
        if np.abs(term[1]) > 1e-6:
            terms_set.append(act_idx(term))
    return set(terms_set[0]).intersection(*terms_set[1:])

unique_ele(op_pro), unique_ele(op_fund)

({1, 2}, set())

In [50]:
np.linalg.eigvals(rho_1)

array([0.97351211, 0.92118047, 0.31511585, 0.31511585, 0.79019157,
       0.79019157, 1.        , 0.97351211, 0.92118047, 1.        ,
       1.        , 1.        ])

In [ ]:
# Verifiquemos que los orbitales son ortogonales
op_arr = np.zeros((basis.d,basis.d),dtype=object)
sp_basis = fixed_basis(basis.d, 1)
for i in tqdm(range(basis.d)):
    for j in range(basis.d):
        oi = np.real(of.get_sparse_operator(targ_sp[i], n_qubits=basis.d))
        oj = np.real(of.get_sparse_operator(targ_sp[j], n_qubits=basis.d))
        # Sin daguear y dagueado
        assert len((oi * oj + oj * oi).data) == 0
        assert len((oi.T * oj.T + oj.T * oi.T).data) == 0
        # Términos cruzados
        op_arr[i,j] = oi * oj.T + oj.T * oi
        sh = op_arr[i,j].shape[0]
        if i != j:
            assert np.allclose(np.zeros((sh,sh)),op_arr[i,j].todense())
        else:
            assert np.allclose(np.eye(sh),op_arr[i,j].todense())

100%|██████████| 12/12 [01:00<00:00,  5.07s/it]


In [32]:
est = 0
app_r = lambda x: np.round(1/(x**2), 0)
for i, ii in enumerate(opt.x):
    if np.abs(ii) > 0.1:
        print(i, ii, basis.base[i])
        if np.abs(ii) > 0.2:
            print(app_r(ii))
            est += 1/np.sqrt(app_r(ii)) * basis.canonicals[i]
            print('')

#(1/np.sqrt(3), 1/np.sqrt(12))
#print(est)
est = 1/np.sqrt(12) * (-basis.canonicals[5]+basis.canonicals[25]+basis.canonicals[85]-basis.canonicals[105])+1/np.sqrt(3)*(basis.canonicals[45]+basis.canonicals[65])

print(r_eig(rho_m(est, rho_2_arrays)))


5 -0.27936976320734497 1.0 [0^ 1^ 2^ 3^ 4^ 7^ 8^ 9^ 12^ 13^]
13.0

25 0.28996316812175205 1.0 [0^ 1^ 2^ 3^ 5^ 7^ 8^ 9^ 11^ 13^]
12.0

45 0.5718530568911167 1.0 [0^ 1^ 2^ 3^ 6^ 7^ 8^ 9^ 11^ 12^]
3.0

65 0.572154480816062 1.0 [0^ 1^ 2^ 4^ 5^ 7^ 8^ 9^ 10^ 13^]
3.0

85 0.2897326805359033 1.0 [0^ 1^ 2^ 4^ 6^ 7^ 8^ 9^ 10^ 12^]
12.0

105 -0.27954194550717076 1.0 [0^ 1^ 2^ 5^ 6^ 7^ 8^ 9^ 10^ 11^]
13.0

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.33333333
 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.5
 0.5        0.5        0.5        0.5        0

In [259]:
r_eig(rho_m(vect, rho_2_arrays))

array([6.39649654e-18, 2.27759896e-17, 2.27759896e-17, 2.97340303e-17,
       1.05847838e-08, 1.05847839e-08, 8.12469946e-02, 8.12469946e-02,
       8.12469946e-02, 8.12469946e-02, 8.12469946e-02, 8.12469946e-02,
       8.26670825e-02, 8.26670825e-02, 8.26670825e-02, 8.26670825e-02,
       8.26670825e-02, 8.26670825e-02, 3.36085923e-01, 3.36085923e-01,
       3.36085923e-01, 3.36085923e-01, 3.36085923e-01, 3.36085923e-01,
       7.48001237e-01, 7.48001237e-01, 7.51998753e-01, 7.51998753e-01])

#### Testing

In [256]:
vect = [-4.646928018967011e-01,2.770294629368069e-13,6.042253100720720e-13,-6.024516357133285e-13,-2.567187786718633e-13,1.437594192304828e-01,-2.763730452330614e-13,4.646928018967681e-01,6.783135520862164e-13,6.755585098225312e-13,-1.437594192304206e-01,2.565069368880155e-13,-6.036534951757788e-13,-6.777043000828106e-13,-6.255919748617283e-14,-2.875188384604817e-01,-6.292320178844996e-13,-5.606330583371935e-13,6.014298958703158e-13,-6.759194003610720e-13,-2.875188384604817e-01,-6.207393804663961e-14,-6.269833844031960e-13,5.590088393571927e-13,2.557458444682729e-13,-1.437594192304207e-01,6.287588334290853e-13,6.270036860946324e-13,4.000749631766127e-01,-2.383049928519640e-13,1.437594192304827e-01,-2.559215089660516e-13,5.600763473792744e-13,-5.580405389843103e-13,2.382296361695720e-13,-4.000749631765536e-01]
vect = np.array(vect)
vect

array([-4.64692802e-01,  2.77029463e-13,  6.04225310e-13, -6.02451636e-13,
       -2.56718779e-13,  1.43759419e-01, -2.76373045e-13,  4.64692802e-01,
        6.78313552e-13,  6.75558510e-13, -1.43759419e-01,  2.56506937e-13,
       -6.03653495e-13, -6.77704300e-13, -6.25591975e-14, -2.87518838e-01,
       -6.29232018e-13, -5.60633058e-13,  6.01429896e-13, -6.75919400e-13,
       -2.87518838e-01, -6.20739380e-14, -6.26983384e-13,  5.59008839e-13,
        2.55745844e-13, -1.43759419e-01,  6.28758833e-13,  6.27003686e-13,
        4.00074963e-01, -2.38304993e-13,  1.43759419e-01, -2.55921509e-13,
        5.60076347e-13, -5.58040539e-13,  2.38229636e-13, -4.00074963e-01])

In [167]:
np.sort(np.linalg.eigvals(rho_m(vect, rho_2_arrays).todense()))

array([6.39649654e-18+0.00000000e+00j, 2.27759896e-17-1.60872919e-17j,
       2.27759896e-17+1.60872919e-17j, 2.97340303e-17+0.00000000e+00j,
       1.05847838e-08+0.00000000e+00j, 1.05847839e-08+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       7.48001237e-01+0.00000000e+00j, 7.48001237e-01+0.00000000e+00j,
       7.51998753e-01+0.00000000e+00j, 7.51998753e-01+0.00000000e+00j])